# **Complementary material for the Frugal AI PSL week**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/frugal_ai/blob/main/GPU_matmul_demo.ipynb)

This notebook aims at demonstrating the performances of matrix multiplication operations on GPUs using the optimized cuBLAS (CUDA BLAS library). We also demonstrate how a naive matrix multiply kernel can be coded, and present a more complex custom implementation (inspired by cuTLASS design) to illustrate the type of low level optimization that are possible in GPGPU CUDA programming.

This notebook is to be use as a complement to the practical work session on matmul optimization on CPU from the Frugal AI PSL week, therefore the provided codes follows a similar strucutre.

## **CuBLAS matmul**

In [ ]:
%%writefile matmul_cublas.cu

#include <stdio.h>
#include <stdlib.h>
#include <cublas_v2.h>

int main(int argc, char *argv[])
{
    size_t l_size = atoi(argv[1]);
    size_t i;
    int M, N, K;
    float *A, *B, *C;
    float alpha = 1.0f, beta = 0.0f;
    M = l_size; N = l_size; K = l_size;

    // Initializing cuBLAS by creating a handle and defining the precision (here pedantic = FP32)
    // Only need to be done once, then all matmul operation can use the same handle
    cublasHandle_t handle;
    cublasStatus_t cublasStat = cublasCreate(&handle);
    cublasSetMathMode(handle, CUBLAS_PEDANTIC_MATH);

    // Allocate and fill the matrices on the host memory
    A = (float*) calloc(M*K,sizeof(float));
    B = (float*) calloc(K*N,sizeof(float));
    C = (float*) calloc(M*N,sizeof(float));

    for(i = 0; i < M*K; i++)
      A[i] = ((float)rand()/RAND_MAX-0.5)*0.1;

    for(i = 0; i < K*N; i++)
      B[i] = ((float)rand()/RAND_MAX-0.5)*0.1;

    // Allocate memory on the GPU to store the matrices
    float *cu_A, *cu_B, *cu_C;

    cudaMalloc(&cu_A, M*K*sizeof(float));
    cudaMalloc(&cu_B, K*N*sizeof(float));
    cudaMalloc(&cu_C, M*N*sizeof(float));

    // Copy the matrices content from the host memory to the allocated sapce in GPU memory
    cudaMemcpy(cu_A, A, M*K*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(cu_B, B, K*N*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(cu_C, C, M*N*sizeof(float), cudaMemcpyHostToDevice);

    // CUDA kernels are asynchronous, so they cannot be timed with regulier host timers
    // We rely on the dedicated "cudaEvent" to measure computation time
    cudaEvent_t start, stop;
    float elapsed_time = 0;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Actual call to CuBLAS Sgemm function (timed with event records)
    cudaEventRecord(start);
    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, M, N, K, &alpha, cu_A, M, cu_B, K, &beta, cu_C, M);
    cudaEventRecord(stop);

    //Get back the result matrix C content from the GPU to the host memory
    cudaMemcpy(C, cu_C, M*N*sizeof(float), cudaMemcpyDeviceToHost);

    // Wait for asynchronous record over "stop" to terminate
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);

    printf("%f", elapsed_time/1000.0); //Convert back in seconds

    exit(EXIT_SUCCESS);
}


Overwriting matmul_cublas.cu


In [ ]:
import os, sys

# Invoke the cuda compiler with linking to the cuBLAS library
# arch_sm is a flag specific to the used GPU architeture
if os.system("nvcc -O3 -arch=sm_75 matmul_cublas.cu -o matmul_cublas -lm -lcublas -lcudart"): sys.exit("Compilation error")

l_size = 4096

elapsed_time_cublas_sgemm = float(os.popen("./matmul_cublas %d"%(l_size)).read())
gflops_cublas_sgemm = l_size**3/1e9/elapsed_time_cublas_sgemm
print ("cuBLAS SGEMM at size %d \t time %f s \t GFLOPS %f"%(l_size, elapsed_time_cublas_sgemm, gflops_cublas_sgemm))

cuBLAS SGEMM at size 16384 	 time 2.565427 s 	 GFLOPS 1714.352625


## **Naiv CUDA kernel**

Here we try to write the simplest possible kernel to parallelize matmul over the GPU using a classical 3-loop implementation. In CUDA programming, the kernel is execute by each CUDA thread, following the SIMT (Single Instruction Multiple Threads) formalism. The simplest way of coding a naive matmul is therefore to have each thread compute one element of the matrix $C$, by computing the loop over $K$. As we learned from the CPU practical work that memory continuity is key, we will adopt the formalism of matmul v3 and also include the transposition of matrix A prior to moving it to the GPU memory and calling the kernel.

This is what is illustrated in the following cell.

In [ ]:
%%writefile matmul_cuda_naive.cu

#include <stdio.h>
#include <stdlib.h>


__global__ void naive_cuda_matmul(float *A_T, float *B, float *C, int M, int N, int K, size_t l_size)
{
    int c_id = blockIdx.x * blockDim.x + threadIdx.x;

    if(c_id >= l_size)
      return;

    int col = c_id / M;
    int row = c_id % M;
    float acc = 0.0;

    for(int k=0; k < K; k++)
      acc += A_T[row*K + k] * B[col*N + k];

    C[c_id] = acc;
}


int main(int argc, char *argv[])
{
    size_t l_size = atoi(argv[1]);
    size_t i, k;
    int M, N, K;
    float *A, *A_T, *B, *C;
    M = l_size; N = l_size; K = l_size;

    // Allocate and fill the matrices on the host memory
    A = (float*) calloc(M*K,sizeof(float));
    A_T = (float*) calloc(M*K,sizeof(float));
    B = (float*) calloc(K*N,sizeof(float));
    C = (float*) calloc(M*N,sizeof(float));

    for(i = 0; i < M*K; i++)
      A[i] = sqrt(i)*0.0001;//((float)rand()/RAND_MAX-0.5)*0.1;

    for(i = 0; i < K*N; i++)
      B[i] = sqrt(i)*0.0001; //((float)rand()/RAND_MAX-0.5)*0.1;

    //Transpose in advance in the CPU
    for(i = 0; i < M; i++)
        for(k = 0; k < K; k++)
            A_T[i*K+k] = A[k*M+i];

    // Allocate memory on the GPU to store the matrices
    float *cu_A_T, *cu_B, *cu_C;

    cudaMalloc(&cu_A_T, M*K*sizeof(float));
    cudaMalloc(&cu_B, K*N*sizeof(float));
    cudaMalloc(&cu_C, M*N*sizeof(float));

    // Copy the matrices content from the host memory to the allocated sapce in GPU memory
    cudaMemcpy(cu_A_T, A_T, M*K*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(cu_B, B, K*N*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(cu_C, C, M*N*sizeof(float), cudaMemcpyHostToDevice);

    // CUDA kernels are asynchronous, so they cannot be timed with regulier host timers
    // We rely on the dedicated "cudaEvent" to measure computation time
    cudaEvent_t start, stop;
    float elapsed_time = 0;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // CUDA kernels are launched as logical thread blocks
    // Here we define how we want to distribute the work over the C matrix

    int cu_threads = 256;
  	int cu_blocks = (M * N + cu_threads - 1) / cu_threads;

    // Actual call to the matmul function/kernel (timed with event records)
    cudaEventRecord(start);
    naive_cuda_matmul<<< cu_blocks, cu_threads >>>(cu_A_T, cu_B, cu_C, M, N, K, M*N);
    cudaEventRecord(stop);

    //Get back the result matrix C content from the GPU to the host memory
    cudaMemcpy(C, cu_C, M*N*sizeof(float), cudaMemcpyDeviceToHost);

    // Wait for asynchronous record over "stop" to terminate
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);

    printf("%f", elapsed_time/1000.0); //Convert back in seconds

    exit(EXIT_SUCCESS);
}

Overwriting matmul_cuda_naive.cu


In [ ]:
import os, sys

# Invoke the cuda compiler with linking to the cuBLAS library
# arch_sm is a flag specific to the used GPU architeture
if os.system("nvcc -O3 -arch=sm_75 matmul_cuda_naive.cu -o matmul_cuda_naive -lm"): sys.exit("Compilation error")

l_size = 4096

elapsed_time_cuda_naive = float(os.popen("./matmul_cuda_naive %d"%(l_size)).read())
gflops_cuda_naive = l_size**3/1e9/elapsed_time_cuda_naive
print ("Matmul CUDA naive at size %d \t time %f s \t GFLOPS %f"%(l_size, elapsed_time_cuda_naive, gflops_cuda_naive))

Matmul CUDA naive at size 8192 	 time 34.941535 s 	 GFLOPS 15.733591


## **Custom implementation**

Following [cuTLASS matmul strategy](https://developer.nvidia.com/blog/cutlass-linear-algebra-cuda/), the following cell propose a custom implementation that includes global matrix blocking, thread block tiling, and warp tiling. This optimize memory reuse at the level of per-thread-block shared memory and register memory at the warp level.

This custom version reaches near 60% of the compute performances of the classical cuBLAS sgemm imlplementation on a T4 GPU depending the matrix size. The tiling size being fixed the performances can vary strongly from on GPU to the other.

In [ ]:
%%writefile matmul_cuda_custom_sgemm.cu

#include <stdio.h>
#include <stdlib.h>


#define BlockDimM 64
#define BlockDimN 64
#define BlockDimK 4

#define WarpDimM 32
#define WarpDimN 16
#define WarpDimK 1

#define ThreadDimM 4
#define ThreadDimN 4


__global__ void custom_sgemm(int TransA, int TransB,
    int M, int N, int K,
    float alpha,
    float *A, int lda,
    float *B, int ldb,
    float beta,
    float *C, int ldc)
{

    int block_row = blockIdx.x;
    int block_col = blockIdx.y;

    float frag_a[ThreadDimM];
    float frag_b[ThreadDimN];

    float accumulator[ThreadDimN][ThreadDimM];

    int c_id = threadIdx.y*blockDim.x + threadIdx.x;
    int warp_id = c_id / 32;
    int thread_id = c_id % 32;

    int warp_x = warp_id / (BlockDimM/WarpDimM);
    int warp_y = warp_id % (BlockDimM/WarpDimM);

    int frag_x = thread_id / (WarpDimM/ThreadDimM);
    int frag_y = thread_id % (WarpDimM/ThreadDimM);

    float *data_block_A;
    float *data_block_B;
    float *data_block_C;

    __shared__ float As[BlockDimK][BlockDimM];
    __shared__ float Bs[BlockDimN][BlockDimK];

    int l, e_per_th, p_x, p_y;

    data_block_C = &C[block_col*BlockDimN*M + block_row*BlockDimM];

    #pragma unroll
    for(int thread_x = 0; thread_x < ThreadDimN; thread_x ++)
      #pragma unroll
      for(int thread_y = 0; thread_y < ThreadDimM; thread_y++)
        accumulator[thread_x][thread_y] = 0.0f;

    for(int block_k = 0; block_k < (K/BlockDimK); block_k++)
    {
      data_block_A = &A[block_k*BlockDimK*M + block_row*BlockDimM];
      data_block_B = &B[block_col*BlockDimN*K + block_k*BlockDimK];

      e_per_th = (BlockDimM*BlockDimK)/(blockDim.x*blockDim.y);
      for(l = 0; l < e_per_th; l++)
      {
        p_x = (c_id*e_per_th + l)/BlockDimM;
        p_y = (c_id*e_per_th + l)%BlockDimM;

        As[p_x][p_y] = data_block_A[p_x*M + p_y];
      }

      __syncthreads();

      e_per_th = (BlockDimN*BlockDimK)/(blockDim.x*blockDim.y);
      for(l = 0; l < e_per_th; l++)
      {
        p_x = (c_id*e_per_th + l)/BlockDimK;
        p_y = (c_id*e_per_th + l)%BlockDimK;

        Bs[p_x][p_y] = data_block_B[p_x*K + p_y];
      }

      __syncthreads();

      #pragma unroll
      for(int warp_k = 0; warp_k < BlockDimK; warp_k++)
      {
        #pragma unroll
        for(int thread_y = 0; thread_y < ThreadDimM; thread_y++)
          frag_a[thread_y] = As[warp_k][warp_y*WarpDimM + frag_y*ThreadDimM + thread_y];
        #pragma unroll
        for(int thread_x = 0; thread_x < ThreadDimN; thread_x ++)
          frag_b[thread_x] = Bs[warp_x*WarpDimN + frag_x*ThreadDimN + thread_x][warp_k];

        #pragma unroll
        for(int thread_x = 0; thread_x < ThreadDimN; thread_x ++)
          #pragma unroll
          for(int thread_y = 0; thread_y < ThreadDimM; thread_y++)
            accumulator[thread_x][thread_y] +=
              frag_a[thread_y]*frag_b[thread_x];
      }
      __syncthreads();
    }

    #pragma unroll
    for(int thread_x = 0; thread_x < ThreadDimN; thread_x ++)
    {

      #pragma unroll
      for(int thread_y = 0; thread_y < ThreadDimM; thread_y++)

        data_block_C[(warp_x*WarpDimN + frag_x*ThreadDimN + thread_x)*M + (warp_y*WarpDimM + frag_y*ThreadDimM + thread_y)] =
          alpha*accumulator[thread_x][thread_y] + beta*data_block_C[(warp_x*WarpDimN + frag_x*ThreadDimN + thread_x)*M + (warp_y*WarpDimM + frag_y*ThreadDimM + thread_y)] ;
    }

}


int main(int argc, char *argv[])
{
    size_t l_size = atoi(argv[1]);
    size_t i;
    int M, N, K;
    float *A, *B, *C;
    M = l_size; N = l_size; K = l_size;

    // Allocate and fill the matrices on the host memory
    A = (float*) calloc(M*K,sizeof(float));
    B = (float*) calloc(K*N,sizeof(float));
    C = (float*) calloc(M*N,sizeof(float));

    for(i = 0; i < M*K; i++)
      A[i] = sqrt(i)*0.0001;//((float)rand()/RAND_MAX-0.5)*0.1;

    for(i = 0; i < K*N; i++)
      B[i] = sqrt(i)*0.0001; //((float)rand()/RAND_MAX-0.5)*0.1;

    // Allocate memory on the GPU to store the matrices
    float *cu_A, *cu_B, *cu_C;

    cudaMalloc(&cu_A, M*K*sizeof(float));
    cudaMalloc(&cu_B, K*N*sizeof(float));
    cudaMalloc(&cu_C, M*N*sizeof(float));

    // Copy the matrices content from the host memory to the allocated sapce in GPU memory
    cudaMemcpy(cu_A, A, M*K*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(cu_B, B, K*N*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(cu_C, C, M*N*sizeof(float), cudaMemcpyHostToDevice);

    // CUDA kernels are asynchronous, so they cannot be timed with regulier host timers
    // We rely on the dedicated "cudaEvent" to measure computation time
    cudaEvent_t start, stop;
    float elapsed_time = 0;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // CUDA kernels are launched as logical thread blocks
    // Here we define how we want to distribute the work over the C matrix

    dim3 threadsPerBlock(BlockDimM/ThreadDimM, BlockDimN/ThreadDimN);
    dim3 numBlocks(M/BlockDimM, N/BlockDimN);

    // Actual call to the matmul function/kernel (timed with event records)
    cudaEventRecord(start);
    custom_sgemm<<< numBlocks, threadsPerBlock >>>(0, 0, M, N, K, 1.0, cu_A, M, cu_B, K, 0.0, cu_C, M);
    cudaEventRecord(stop);

    //Get back the result matrix C content from the GPU to the host memory
    cudaMemcpy(C, cu_C, M*N*sizeof(float), cudaMemcpyDeviceToHost);

    // Wait for asynchronous record over "stop" to terminate
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);

    printf("%f", elapsed_time/1000.0); //Convert back in seconds

    exit(EXIT_SUCCESS);
}

Overwriting matmul_cuda_custom_sgemm.cu


In [ ]:
# Invoke the cuda compiler with linking to the cuBLAS library
# arch_sm is a flag specific to the used GPU architeture
if os.system("nvcc -O3 -arch=sm_75 matmul_cuda_custom_sgemm.cu -o matmul_cuda_custom_sgemm -lm"): sys.exit("Compilation error")

l_size = 4096

elapsed_time_custom_sgemm = float(os.popen("./matmul_cuda_custom_sgemm %d"%(l_size)).read())
gflops_custom_sgemm = l_size**3/1e9/elapsed_time_custom_sgemm
print ("Matmul CUDA naive at size %d \t time %f s \t GFLOPS %f"%(l_size, elapsed_time_custom_sgemm, gflops_custom_sgemm))

Matmul CUDA naive at size 16384 	 time 5.202236 s 	 GFLOPS 845.414647
